# Insurance cost prediction using linear regression

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from: https://www.kaggle.com/mirichoi0218/insurance


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lectures. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ml/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ml/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ml/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ml/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ml/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [1]:
# Uncomment and run the commands below if imports fail
# !conda install numpy pytorch torchvision cpuonly -c pytorch -y
# !pip install matplotlib --upgrade --quiet
# !pip install jovian --upgrade --quiet

In [2]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

<IPython.core.display.Javascript object>

In [3]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [4]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: .\insurance.csv


To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [5]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [6]:
your_name = "Praveen Raghuvanshi" # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [7]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [8]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,region,charges
240,23,female,41.8038,2,yes,northeast,37356.279451
651,53,female,45.1440,1,no,southeast,10262.319670
761,23,male,40.1280,1,no,southwest,2344.446350
836,36,male,35.9100,0,no,southwest,4270.166010
1147,20,female,36.3888,0,no,northwest,2193.721736


In [10]:
dataframe.isnull().count()

age         1271
sex         1271
bmi         1271
children    1271
smoker      1271
region      1271
charges     1271
dtype: int64

In [11]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1271 entries, 240 to 1280
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1271 non-null   int64  
 1   sex       1271 non-null   object 
 2   bmi       1271 non-null   float64
 3   children  1271 non-null   int64  
 4   smoker    1271 non-null   object 
 5   region    1271 non-null   object 
 6   charges   1271 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 79.4+ KB


In [12]:
dataframe.describe()

,age,bmi,children,charges
count,1271.000000,1271.000000,1271.000000,1271.000000
mean,39.370574,34.969540,1.104642,12972.222716
std,14.040786,6.986546,1.213144,11811.899486
min,18.000000,18.194400,0.000000,1088.217683
25%,27.000000,29.999100,0.000000,4629.627785
50%,40.000000,34.656000,1.000000,9163.832840
75%,51.000000,39.432600,2.000000,16282.767255
max,64.000000,60.568200,5.000000,61857.315170


In [13]:
dataframe.isnull().values.any()

False

Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [14]:
num_rows = dataframe.shape[0]
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [15]:
num_cols = dataframe.shape[1]
print(num_cols)

7


**Q: What are the column titles of the input variables?**

In [16]:
output_col_index = dataframe.columns.size - 1
input_cols = list(dataframe.columns[0:output_col_index])
print(input_cols)

['age', 'sex', 'bmi', 'children', 'smoker', 'region']


**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [17]:
categorical_cols = dataframe.select_dtypes(include='object').columns.to_list()
print(categorical_cols)

['sex', 'smoker', 'region']


**Q: What are the column titles of output/target variable(s)?**

In [18]:
output_cols = list(dataframe.columns[output_col_index:])
print(output_cols)

['charges']


**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [19]:
# Write your answer here
dataframe[output_cols].describe()

,charges
count,1271.000000
mean,12972.222716
std,11811.899486
min,1088.217683
25%,4629.627785
50%,9163.832840
75%,16282.767255
max,61857.315170


Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [20]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "praveenraghuvanshi1512/02-insurance-linear-regression" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Committed successfully! https://jovian.ml/praveenraghuvanshi1512/02-insurance-linear-regression


'https://jovian.ml/praveenraghuvanshi1512/02-insurance-linear-regression'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [21]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy(dtype='float32')
    targets_array = dataframe1[output_cols].to_numpy(dtype='float32')
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [22]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[23.    ,  0.    , 41.8038,  2.    ,  1.    ,  0.    ],
        [53.    ,  0.    , 45.144 ,  1.    ,  0.    ,  2.    ],
        [23.    ,  1.    , 40.128 ,  1.    ,  0.    ,  3.    ],
        ...,
        [59.    ,  0.    , 41.9121,  1.    ,  1.    ,  0.    ],
        [53.    ,  0.    , 37.905 ,  0.    ,  0.    ,  0.    ],
        [48.    ,  0.    , 37.9962,  0.    ,  0.    ,  2.    ]],
       dtype=float32),
 array([[37356.28  ],
        [10262.319 ],
        [ 2344.4463],
        ...,
        [46459.887 ],
        [10247.9375],
        [ 8035.1704]], dtype=float32))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [23]:
inputs = torch.from_numpy(inputs_array)
targets = torch.from_numpy(targets_array)

In [24]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [25]:
dataset = TensorDataset(inputs, targets)

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets. **

In [26]:
val_percent = 0.15 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset, [train_size, val_size]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [27]:
batch_size = 4

In [28]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [29]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[62.0000,  1.0000, 36.4971,  0.0000,  1.0000,  0.0000],
        [48.0000,  1.0000, 39.1020,  3.0000,  0.0000,  3.0000],
        [20.0000,  1.0000, 35.4882,  2.0000,  0.0000,  2.0000],
        [37.0000,  0.0000, 35.0892,  0.0000,  1.0000,  0.0000]])
targets: tensor([[44338.9023],
        [ 9276.1377],
        [ 2489.4766],
        [36152.0469]])


Let's save our work by committing to Jovian.

In [30]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "praveenraghuvanshi1512/02-insurance-linear-regression" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Committed successfully! https://jovian.ml/praveenraghuvanshi1512/02-insurance-linear-regression


'https://jovian.ml/praveenraghuvanshi1512/02-insurance-linear-regression'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [31]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [32]:
import torch.nn.functional as F

In [33]:
class InsuranceModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                  # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        # loss = F.mse_loss(out, targets)                          # fill this
        self.criterion  = nn.SmoothL1Loss()
        loss = self.criterion(out,targets)
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        # loss = F.mse_loss(out, targets)                           # fill this    
        self.criterion  = nn.SmoothL1Loss()
        loss = self.criterion(out,targets)
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [34]:
model = InsuranceModel(input_size, output_size)

Let's check out the weights and biases of the model using `model.parameters`.

In [35]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1888, -0.2477, -0.0947, -0.0083,  0.1502, -0.0669]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.3538], requires_grad=True)]

One final commit before we train the model.

In [36]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "praveenraghuvanshi1512/02-insurance-linear-regression" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Committed successfully! https://jovian.ml/praveenraghuvanshi1512/02-insurance-linear-regression


'https://jovian.ml/praveenraghuvanshi1512/02-insurance-linear-regression'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [37]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [38]:
result = evaluate(model, val_loader) # Use the the evaluate function
print(result)

{'val_loss': 13096.9326171875}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [39]:
epochs = 100
lr = 0.1
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 7001.2378
Epoch [40], val_loss: 6980.1367
Epoch [60], val_loss: 6886.4624
Epoch [80], val_loss: 6857.9902
Epoch [100], val_loss: 6891.2505


In [40]:
epochs = 100
lr = 0.01
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6829.3145
Epoch [40], val_loss: 6828.9028
Epoch [60], val_loss: 6824.0571
Epoch [80], val_loss: 6825.2085
Epoch [100], val_loss: 6828.1694


In [41]:
epochs = 100
lr = 0.001
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6818.9907
Epoch [40], val_loss: 6819.6489
Epoch [60], val_loss: 6818.5605
Epoch [80], val_loss: 6818.2441
Epoch [100], val_loss: 6818.1255


In [42]:
epochs = 100
lr = 0.0001
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6818.2886
Epoch [40], val_loss: 6818.3950
Epoch [60], val_loss: 6818.3599
Epoch [80], val_loss: 6818.3706
Epoch [100], val_loss: 6818.3521


In [43]:
epochs = 100
lr = 0.00001
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6818.3633
Epoch [40], val_loss: 6818.3706
Epoch [60], val_loss: 6818.3755
Epoch [80], val_loss: 6818.3755
Epoch [100], val_loss: 6818.3862


**Q: What is the final validation loss of your model?**

In [44]:
val_loss = 6818.3862

Let's log the final validation loss to Jovian and commit the notebook

In [45]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [46]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "praveenraghuvanshi1512/02-insurance-linear-regression" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ml/praveenraghuvanshi1512/02-insurance-linear-regression


'https://jovian.ml/praveenraghuvanshi1512/02-insurance-linear-regression'

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [47]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [48]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([46.0000,  1.0000, 34.7643,  3.0000,  1.0000,  1.0000])
Target: tensor([39498.9336])
Prediction: tensor([10190.3984])


In [49]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([24.0000,  0.0000, 34.4394,  3.0000,  0.0000,  1.0000])
Target: tensor([4479.5376])
Prediction: tensor([4413.9058])


In [50]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([28.0000,  0.0000, 32.9232,  1.0000,  0.0000,  0.0000])
Target: tensor([4207.6030])
Prediction: tensor([5104.0879])


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to clean up & replicate this notebook (or [this one](https://jovian.ml/aakashns/housing-linear-minimal), or [this one](https://jovian.ml/aakashns/mnist-logistic-minimal) ) for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patters in machine learning from problem-specific details.

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ml/forum/t/share-your-work-here-assignment-2/4931

In [ ]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "praveenraghuvanshi1512/02-insurance-linear-regression" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ml/praveenraghuvanshi1512/02-insurance-linear-regression


<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..


In [114]:
from utils import plot_history

plot_history(history)

ModuleNotFoundError: No module named 'utils'